# Initial Exploration of Non-Image Data from Yelp Dataets

In [40]:
# imports

# dataframe
import pandas as pd
import json
import sqlite3

# NLP
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

In [3]:
# open .json files

# business.json
with open('/Users/ianforrest/Desktop/coding/repos/yelp/yelp_dataset/business.json') as f:
    business = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")

# checkin.json
with open('/Users/ianforrest/Desktop/coding/repos/yelp/yelp_dataset/checkin.json') as f:
    checkin = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")
    
# review.json
with open('/Users/ianforrest/Desktop/coding/repos/yelp/yelp_dataset/review.json') as f:
    review = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")

# tip.json
with open('/Users/ianforrest/Desktop/coding/repos/yelp/yelp_dataset/tip.json') as f:
    tip = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")
    
# user.json
with open('/Users/ianforrest/Desktop/coding/repos/yelp/yelp_dataset/user.json') as f:
    user = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")

In [4]:
# convert .json files to pandas DataFrames
df_business = pd.DataFrame(business)
df_checkin = pd.DataFrame(checkin)
df_review = pd.DataFrame(review)
df_tip = pd.DataFrame(tip)
df_user = pd.DataFrame(user)

In [5]:
# view df_business
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,2818 E Camino Acequia Drive,Phoenix,AZ,85016,33.522143,-112.018481,3.0,5,0,{'GoodForKids': 'False'},"Golf, Active Life",None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,1,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,1,"{'GoodForKids': 'True', 'NoiseLevel': 'u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-..."
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,"15655 W Roosevelt St, Ste 237",Goodyear,AZ,85338,33.455613,-112.395596,5.0,3,1,None,"Insurance, Financial Services","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,"4209 Stuart Andrew Blvd, Ste F",Charlotte,NC,28217,35.190012,-80.887223,4.0,4,1,"{'BusinessAcceptsBitcoin': 'False', 'ByAppoint...","Plumbing, Shopping, Local Services, Home Servi...","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


In [14]:
df_business.shape

(192609, 14)

In [23]:
df_business = pd.concat([df_business.drop(['attributes'], axis=1), df_business['attributes'].apply(pd.Series)], axis=1)

In [6]:
# view df_checkin
df_checkin.head()

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."


In [15]:
df_checkin.shape

(161950, 2)

In [7]:
# view df_review
df_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [8]:
df_review.shape

(6685900, 9)

In [9]:
# check df_tip
df_tip.head()

,user_id,business_id,text,date,compliment_count
0,UPw5DWs_b-e2JRBS-t37Ag,VaKXUpmWTTWDKbpJ3aQdMw,"Great for watching games, ufc, and whatever el...",2014-03-27 03:51:24,0
1,Ocha4kZBHb4JK0lOWvE0sg,OPiPeoJiv92rENwbq76orA,Happy Hour 2-4 daily with 1/2 price drinks and...,2013-05-25 06:00:56,0
2,jRyO2V1pA4CdVVqCIOPc1Q,5KheTjYPu1HcQzQFtm4_vw,Good chips and salsa. Loud at times. Good serv...,2011-12-26 01:46:17,0
3,FuTJWFYm4UKqewaosss1KA,TkoyGi8J7YFjA6SbaRzrxg,The setting and decoration here is amazing. Co...,2014-03-23 21:32:49,0
4,LUlKtaM3nXd-E4N4uOk_fQ,AkL6Ous6A1atZejfZXn1Bg,Molly is definately taking a picture with Sant...,2012-10-06 00:19:27,0


In [10]:
df_tip.shape

(1223094, 5)

In [16]:
df_tip.groupby(["user_id", "business_id"]).size()

user_id                 business_id           
---1lKK3aKOuomHnwAkAow  0zoXYHq82haayMaV952jEQ    1
                        2VNa2kbbt4o8nQmPKIcHVQ    1
                        3noEVWVC1vckcnvG_wghxA    1
                        4RoTEeqB_MNn6yaqZmlZHg    1
                        5rxJpTkeJa5rxMvL2NbSnQ    1
                                                 ..
zzvm_VD3HjU6RMx8zq7g1g  110iMPMPEEjFlf8HKVq84g    1
                        V6MWAv-xv4dgSUrDU1qxaA    1
zzyMMeUZzKAy7KQhM7lU2w  N7yuiiu8jhQ-Fl9Npflreg    1
                        xsdcF72nqAhlYuKs45KZUw    1
zzyzP--p98zZtx359NvgGA  7sGcBZwkD4yFLiBLS1WKgA    1
Length: 1033713, dtype: int64

In [11]:
# check df_user
df_user.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,l6BmjZMeQD3rDxWUbiAiow,Rashmi,95,2013-10-08 23:11:33,84,17,25,"2015,2016,2017","c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g...",5,...,0,0,0,0,1,1,1,1,2,0
1,4XChL029mKr5hydo79Ljxg,Jenna,33,2013-02-21 22:29:06,48,22,16,,"kEBTgDvFX754S68FllfCaA, aB2DynOxNOJK9st2ZeGTPg...",4,...,0,0,0,0,0,0,1,1,0,0
2,bc8C_eETBWL0olvFSJJd0w,David,16,2013-10-04 00:16:10,28,8,10,,"4N-HU_T32hLENLntsNKNBg, pSY2vwWLgWfGVAAiKQzMng...",0,...,0,0,0,0,1,0,0,0,0,0
3,dD0gZpBctWGdWo9WlGuhlA,Angela,17,2014-05-22 15:57:30,30,4,14,,"RZ6wS38wnlXyj-OOdTzBxA, l5jxZh1KsgI8rMunm-GN6A...",5,...,0,0,0,0,0,2,0,0,1,0
4,MM4RJAeH6yuaN8oZDSt0RA,Nancy,361,2013-10-23 07:02:50,1114,279,665,"2015,2016,2017,2018","mbwrZ-RS76V1HoJ0bF_Geg, g64lOV39xSLRZO0aQQ6DeQ...",39,...,1,0,0,1,16,57,80,80,25,5


In [13]:
df_user.shape

(1637138, 22)

In [24]:
# create SQLite database to create final dataframe
conn = sqlite3.connect('/Users/ianforrest/Desktop/coding/repos/yelp/db.sqlite')

In [25]:
# export review, tip, and user data as individual tables to sqlite database

df_tip.to_sql('tip', conn)
df_user.to_sql('user', conn)
df_review.to_sql('review', conn)

In [36]:
# create new dataframe merging df_review with df_user
# adding user data to each individual review
df = df_review.merge(df_user, how='left', left_on='user_id',right_on='user_id')

In [37]:
df.shape

(6685900, 30)

In [38]:
df.head()

,review_id,user_id,business_id,stars,useful_x,funny_x,cool_x,text,date,name,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,Stefan,...,0,0,0,0,0,0,0,0,0,0
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,Jackie,...,2,1,1,0,7,12,10,10,2,1
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03,Wilhelmina,...,0,0,0,0,1,1,0,0,0,0
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,Simon,...,0,0,0,0,0,0,0,0,0,0
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,Reilly,...,0,0,0,0,0,0,0,0,0,0


In [39]:
df.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful_x', 'funny_x',
       'cool_x', 'text', 'date', 'name', 'review_count', 'yelping_since',
       'useful_y', 'funny_y', 'cool_y', 'elite', 'friends', 'fans',
       'average_stars', 'compliment_hot', 'compliment_more',
       'compliment_profile', 'compliment_cute', 'compliment_list',
       'compliment_note', 'compliment_plain', 'compliment_cool',
       'compliment_funny', 'compliment_writer', 'compliment_photos'],
      dtype='object')

In [41]:
# NLP - initiate STOPWORDS; 

STOPWORDS = set(STOPWORDS).union(set(['I', 'We', 'i', 'we', 'it', "it's", 'it.', 'the', 'this', 'they']))

In [43]:
# create tokenize function to tokenize review text
def tokenize(text):
    return [token for token in simple_preprocess(text, deacc=True, min_len=4, max_len=20) if token not in STOPWORDS]

In [44]:
# add tokens column to dataframe 
df['tokens'] = df['text'].apply(tokenize)

In [45]:
df.head()

,review_id,user_id,business_id,stars,useful_x,funny_x,cool_x,text,date,name,...,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,tokens
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,Stefan,...,0,0,0,0,0,0,0,0,0,"[total, horrible, service, crooks, actually, n..."
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,Jackie,...,1,1,0,7,12,10,10,2,1,"[adore, travis, hard, rock, kelly, cardenas, s..."
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03,Wilhelmina,...,0,0,0,1,1,0,0,0,0,"[office, organized, friendly, phillipp, great,..."
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,Simon,...,0,0,0,0,0,0,0,0,0,"[went, lunch, steak, sandwich, delicious, caes..."
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,Reilly,...,0,0,0,0,0,0,0,0,0,"[today, second, sessions, paid, session, went,..."
